In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing libraries
import numpy as np
import os
import random
import cv2
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
import time


In [ ]:
DIRECTORY = '/kaggle/input/dog-vs-cat-fastai/dogscats/train/'
CATEGORIES = ['cats', 'dogs']
IMG_SIZE = 100  # Resize all images to 100x100 pixels for uniformity


In [ ]:
data = []

for category in CATEGORIES:
    folder = os.path.join(DIRECTORY, category)
    label = CATEGORIES.index(category)
    
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_array = cv2.imread(img_path)
        img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        data.append([img_array, label])

random.shuffle(data)


In [ ]:
X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255.0  # Normalization
y = np.array(y)

with open("X.pkl", "wb") as f:
    pickle.dump(X, f)

with open("y.pkl", "wb") as f:
    pickle.dump(y, f)


In [ ]:
with open("X.pkl", "rb") as f:
    X = pickle.load(f)

with open("y.pkl", "rb") as f:
    y = pickle.load(f)


In [ ]:
NAME = f"cat-vs-dog-prediction-{int(time.time())}"
tensorboard = TensorBoard(log_dir=f"logs/{NAME}")


In [ ]:
model = Sequential()

# Convolution and Pooling Layers
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flattening and Dense Layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X, y, epochs=5, batch_size=32, validation_split=0.1, callbacks=[tensorboard])


In [19]:
def prepare_image(filepath):
    img_array = cv2.imread(filepath)
    img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    img_array = img_array / 255.0
    return img_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# Use a test image
test_image_path = r'/kaggle/input/billicat/cat2.jpeg'  # Update this path
prepared_image = prepare_image(test_image_path)
prediction = model.predict(prepared_image)

# Decode the prediction
predicted_class = CATEGORIES[np.argmax(prediction)]
print(f"The model predicts this image is a: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
The model predicts this image is a: cats
